In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras import mixed_precision

# Enable mixed precision

In [ ]:
mixed_precision.set_global_policy("mixed_float16")

# Download the Dataset (15 classes)

In [ ]:

# Load class names from metadata
_, info = tfds.load('food101', split='train', with_info=True)
all_classes = info.features['label'].names

# Define the 15 selected classes
selected_classes = [
    'pizza', 'samosa', 'falafel', 'donuts', 'macaroni_and_cheese',
    'caprese_salad', 'bibimbap', 'ceviche', 'bruschetta', 'beef_carpaccio',
    'gnocchi', 'club_sandwich', 'grilled_cheese_sandwich', 'pad_thai', 'takoyaki'
]
selected_indices = tf.constant([all_classes.index(cls) for cls in selected_classes])

# Filter and remap function
def filter_classes(image, label):
    label = tf.cast(label, tf.int32)
    return tf.reduce_any(tf.equal(label, selected_indices))

def remap_labels(image, label):
    label = tf.cast(label, tf.int32)
    new_label = tf.argmax(tf.cast(tf.equal(label, selected_indices), tf.int32))
    return image, new_label

# Load and filter datasets
training_set = tfds.load('food101', split='train[:90%]', as_supervised=True)
validation_set = tfds.load('food101', split='train[90%:]', as_supervised=True)
testing_set = tfds.load('food101', split='validation', as_supervised=True)

training_set = training_set.filter(filter_classes).map(remap_labels)
validation_set = validation_set.filter(filter_classes).map(remap_labels)
testing_set = testing_set.filter(filter_classes).map(remap_labels)

# Final class names for reference
class_names = selected_classes


# Data Processing

In [ ]:
def preprocess_image(image, label):
    image = tf.image.resize(image, [224, 224]) / 255.0
    return image, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, 0.7, 1.3)
    image = tf.image.random_saturation(image, 0.7, 1.3)
    image = tf.image.random_hue(image, 0.05)
    image = tf.image.resize_with_crop_or_pad(image, 230, 230)
    image = tf.image.random_crop(image, [224, 224, 3])
    return image, label

#preparing datasets
training_set = (training_set
                .map(preprocess_image)
                .map(augment)
                .shuffle(1000)
                .batch(32)
                .prefetch(tf.data.AUTOTUNE))

validation_set = (validation_set
                  .map(preprocess_image)
                  .batch(32)
                  .prefetch(tf.data.AUTOTUNE))

testing_set = (testing_set
               .map(preprocess_image)
               .batch(32)
               .prefetch(tf.data.AUTOTUNE))

In [ ]:
# _, info = tfds.load('food101', split='train', with_info=True)
# class_names = info.features['label'].names[:20]

In [ ]:
batch = next(iter(training_set))
images, labels = batch

images = images.numpy()
labels = labels.numpy()

plt.figure(figsize=(15, 10))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(np.clip(images[i], 0, 1))
    plt.title(class_names[labels[i]])
    plt.axis('off')
plt.show()

# Transfer Learning

In [ ]:
#transfer learning
base_model = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = True

In [ ]:
# import not fine tuned model
model = tf.keras.models.load_model('/content/food_vision_model2.h5')

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = len(base_model.layers) // 2
 
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Fine-tune the model
fine_tune_epochs = 20
 
history_fine = model.fit(
    training_set,
    epochs=fine_tune_epochs,
    validation_data=validation_set
)
 
test_loss, test_acc = model.evaluate(testing_set)
print(f'Test accuracy: {test_acc:.4f}')

In [ ]:
def evaluate_model(model, testing_set, class_names):
    y_true = []
    y_pred = []
 
    for images, labels in testing_set:
        preds = model.predict(images)
        y_true.extend(labels.numpy())
        y_pred.extend(tf.argmax(preds, axis=1).numpy())
 
    report = classification_report(y_true, y_pred, target_names=class_names)
    print(report)

In [ ]:
evaluate_model(model, testing_set, class_names)
 
y_true = []
y_pred = []
for images, labels in testing_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))
 
y_true = np.array(y_true)
y_pred = np.array(y_pred)
 
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix2.png')

In [ ]:

model.save('tuned_food_vision_model.h5')